<a href="https://colab.research.google.com/github/Walidsati/AAI_612O/blob/main/Week6/Notebook6.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AAI612: Deep Learning & its Applications

*Notebook 6.3: Understanding recurrent neural networks*



**Read and parse the data of the Jena weather dataset**

In [9]:
import ssl
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/harmanani/AAI612/main/Week6/data/jena_climate_2009_2016.csv"

ssl._create_default_https_context = ssl._create_unverified_context
f = urlopen(url)
data = f.read()

lines = data.decode().split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(header)
print(len(lines))

import numpy as np
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]]
    temperature[i] = values[1]
    raw_data[i, :] = values[:]

num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420451
num_train_samples: 210225
num_val_samples: 105112
num_test_samples: 105114


**Compute the number of samples we'll use for each data split**

In [10]:
import keras.utils

sampling_rate = 6
sequence_length = 120
delay = sampling_rate * (sequence_length + 24 - 1)
batch_size = 512

train_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)

val_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)

test_dataset = keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples)

**NumPy implementation of a simple RNN**

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
timesteps = 100
input_features = 32
output_features = 64
inputs = np.random.random((timesteps, input_features))
state_t = np.zeros((output_features,))
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features,))
successive_outputs = []
for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    successive_outputs.append(output_t)
    state_t = output_t
final_output_sequence = np.stack(successive_outputs, axis=0)

### A recurrent layer in Keras

**An RNN layer that can process sequences of any length**

In [12]:
num_features = 14
inputs = keras.Input(shape=(None, num_features))
outputs = layers.SimpleRNN(16)(inputs)

**An RNN layer that returns only its last output step**

In [13]:
num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=False)(inputs)
print(outputs.shape)

(None, 16)


**An RNN layer that returns its full output sequence**

In [14]:
num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=True)(inputs)
print(outputs.shape)

(None, 120, 16)


**Stacking RNN layers**

In [15]:
inputs = keras.Input(shape=(steps, num_features))
x = layers.SimpleRNN(16, return_sequences=True)(inputs)
x = layers.SimpleRNN(16, return_sequences=True)(x)
outputs = layers.SimpleRNN(16)(x)

## Advanced use of recurrent neural networks

### Using recurrent dropout to fight overfitting

**Training and evaluating a dropout-regularized LSTM**

In [16]:
sequence_length = 120

inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(32, recurrent_dropout=0.25)(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_lstm_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)

Epoch 1/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 106s 253ms/step - loss: 117.7530 - mae: 8.8426 - val_loss: 75.8443 - val_mae: 7.0571
Epoch 2/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 104s 253ms/step - loss: 78.1801 - mae: 7.0878 - val_loss: 57.4323 - val_mae: 6.1388
Epoch 3/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 104s 253ms/step - loss: 70.5301 - mae: 6.6699 - val_loss: 53.1425 - val_mae: 5.8536
Epoch 4/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 104s 253ms/step - loss: 66.2512 - mae: 6.4268 - val_loss: 50.0681 - val_mae: 5.6839
Epoch 5/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 104s 252ms/step - loss: 62.7985 - mae: 6.2104 - val_loss: 46.3141 - val_mae: 5.4130
Epoch 6/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 104s 253ms/step - loss: 59.6662 - mae: 6.0126 - val_loss: 43.3226 - val_mae: 5.2286
Epoch 7/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 103s 251ms/step - loss: 57.1224 - mae: 5.8556 - val_loss: 41.3767 - val_mae: 5.1413
Epoch 8/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 104s 254ms/step - loss: 55.4098 - mae: 5.7445 - val_loss: 37.3280 - val_mae: 4.7900
Epoch 9

In [17]:
inputs = keras.Input(shape=(sequence_length, num_features))
x = layers.LSTM(32, recurrent_dropout=0.2, unroll=True)(inputs)

### Stacking recurrent layers

**Training and evaluating a dropout-regularized, stacked GRU model**

In [18]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.GRU(32, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(32, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_stacked_gru_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("jena_stacked_gru_dropout.x")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 208s 498ms/step - loss: 68.8074 - mae: 6.4300 - val_loss: 22.6478 - val_mae: 3.7174
Epoch 2/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 203s 494ms/step - loss: 26.1059 - mae: 4.0285 - val_loss: 21.3109 - val_mae: 3.6398
Epoch 3/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 203s 495ms/step - loss: 24.3204 - mae: 3.8990 - val_loss: 22.8520 - val_mae: 3.8103
Epoch 4/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 206s 501ms/step - loss: 23.7358 - mae: 3.8498 - val_loss: 18.8351 - val_mae: 3.3706
Epoch 5/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 205s 500ms/step - loss: 22.9853 - mae: 3.7918 - val_loss: 17.5157 - val_mae: 3.2410
Epoch 6/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 204s 496ms/step - loss: 22.4691 - mae: 3.7540 - val_loss: 18.5257 - val_mae: 3.3560
Epoch 7/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 203s 495ms/step - loss: 22.3842 - mae: 3.7425 - val_loss: 17.2690 - val_mae: 3.2192
Epoch 8/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 203s 495ms/step - loss: 21.9088 - mae: 3.7095 - val_loss: 18.9124 - val_mae: 3.4449
Epoch 9/

ValueError: File format not supported: filepath=jena_stacked_gru_dropout.x. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(jena_stacked_gru_dropout.x, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [20]:
model = keras.models.load_model("jena_stacked_gru_dropout.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

203/203 ━━━━━━━━━━━━━━━━━━━━ 26s 121ms/step - loss: 17.9411 - mae: 3.3118
Test MAE: 3.32


### Using bidirectional RNNs

**Training and evaluating a bidirectional LSTM**

In [19]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.Bidirectional(layers.LSTM(16))(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=val_dataset)

Epoch 1/10
410/410 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 113.6420 - mae: 8.6638 - val_loss: 81.0563 - val_mae: 7.2125
Epoch 2/10
410/410 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 73.8889 - mae: 6.8287 - val_loss: 50.8184 - val_mae: 5.6145
Epoch 3/10
410/410 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 49.4897 - mae: 5.4730 - val_loss: 33.4251 - val_mae: 4.4891
Epoch 4/10
410/410 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 37.8693 - mae: 4.7039 - val_loss: 28.1862 - val_mae: 4.1060
Epoch 5/10
410/410 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 31.9851 - mae: 4.3161 - val_loss: 24.3586 - val_mae: 3.8064
Epoch 6/10
410/410 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 27.7237 - mae: 4.0373 - val_loss: 22.3294 - val_mae: 3.6459
Epoch 7/10
410/410 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 24.7545 - mae: 3.8362 - val_loss: 20.9039 - val_mae: 3.5209
Epoch 8/10
410/410 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 22.6274 - mae: 3.6879 - val_loss: 19.7395 - val_mae: 3.4593
Epoch 9/10
410/410 ━━━━